# Projeto Prático -  AceleraDev Data Science

__Questão de Negócio__:
O objetivo deste produto é fornecer um serviço automatizado que recomenda leads para um usuário dado sua atual lista de clientes (Portfólio).

## 0.0. Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from prince import MCA

%matplotlib inline

### 0.1. Carregando dados

In [233]:
market = pd.read_csv('data/estaticos_market.csv', index_col=0)
portfolio_1 = pd.read_csv('data/estaticos_portfolio1.csv', index_col=0)
portfolio_2 = pd.read_csv('data/estaticos_portfolio2.csv', index_col=0)
portfolio_3 = pd.read_csv('data/estaticos_portfolio3.csv', index_col=0)

## 1.0. Descrição dos dados

### Market

In [226]:
market.head()

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,...,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,14.457534,10 a 15,False,...,43.738462,93.266667,19.166667,26.0,26.0,27.0,0.0,-3.703704,0.0,0
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,SERVICOS DE ALOJAMENTO/ALIMENTACAO,SERVIÇO,1.463014,1 a 5,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,"TRANSPORTE, ARMAZENAGEM E CORREIO",SERVIÇO,7.093151,5 a 10,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,SERVICOS DIVERSOS,SERVIÇO,6.512329,5 a 10,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,True,EMPRESARIO INDIVIDUAL,RN,OUTROS,"SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS",SERVIÇO,3.200000,1 a 5,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [227]:
market.shape

(462298, 181)

In [228]:
market.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 462298 entries, 0 to 462297
Columns: 181 entries, id to qt_filiais
dtypes: bool(9), float64(144), int64(1), object(27)
memory usage: 614.1+ MB


In [243]:
market.duplicated().sum()

0

### Portfolio 1

In [234]:
portfolio_1.head()

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,...,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
0,dabe79bec87c88ae04e869bf6bd321ee5e1893cecf6625...,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,COMERCIO VAREJISTA,COMERCIO,0.646575,<= 1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,32e5f4e10932153a7ba869cb0386e7e02d49d2461046b8...,False,EMPRESARIO INDIVIDUAL,MA,OUTROS,COMERCIO VAREJISTA,COMERCIO,6.586301,5 a 10,False,...,NaN,NaN,NaN,0.0,0.0,2.0,NaN,-100.0,0.0,4
2,a95d6f30bba445bd3d6b0c5b36f865b38ec01d17336090...,True,EMPRESARIO INDIVIDUAL,MA,OUTROS,COMERCIO VAREJISTA,COMERCIO,8.010959,5 a 10,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,6cb309685cea0b6d2988818792ec2e6fcb2bd02e0afa9e...,True,EMPRESARIO INDIVIDUAL,RN,OUTROS,COMERCIO VAREJISTA,COMERCIO,20.863014,> 20,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,f72aa7fa6787b0a5a1c88885b6120850df8ee0f71adc25...,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,COMERCIO VAREJISTA,COMERCIO,18.726027,15 a 20,False,...,39.454762,91.2,19.333333,14.0,15.0,25.0,-6.666667,-44.0,0.0,3


In [235]:
portfolio_1.shape

(555, 181)

In [236]:
portfolio_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 555 entries, 0 to 554
Columns: 181 entries, id to qt_filiais
dtypes: bool(14), float64(142), int64(3), object(22)
memory usage: 736.0+ KB


In [244]:
portfolio_1.duplicated().sum()

0

### Portfolio 2

In [237]:
portfolio_2.head()

,id
0,09e95c1a84049001d086470a8f320a19b076f955a89122...
1,dc9d155f5bcd3172031e2ec9431786e8871ff4b0ff4728...
2,16843c9ffb920170477118933798510d8d5f00f5c56c12...
3,ff045934d90aab184dd15e66daf3b3c97193bda5449298...
4,a0ce528caab2c62497a548d0ebd9e935143ef81ed7dbf0...


In [238]:
portfolio_2.shape

(566, 1)

In [239]:
portfolio_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 566 entries, 0 to 565
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      566 non-null    object
dtypes: object(1)
memory usage: 8.8+ KB


In [245]:
portfolio_2.duplicated().sum()

0

### Portfolio 3

In [240]:
portfolio_3.head()

,id
0,bb61c09449a1c1f8b78955d8b7a7f73599e78358993fe2...
1,16843c9ffb920170477118933798510d8d5f00f5c56c12...
2,1b8d092daff12bed8b3c2ab93bf5df2921865a68aca185...
3,3aafa3a1f12f483740a52956af100223333d0e3c79bf2c...
4,c60431fbca90067f105570e1e8a684bcd7048205bbf6c4...


In [241]:
portfolio_3.shape

(265, 1)

In [242]:
portfolio_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 265 entries, 0 to 264
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      265 non-null    object
dtypes: object(1)
memory usage: 4.1+ KB


In [246]:
portfolio_3.duplicated().sum()

0

Conseguimos notar que, diferentemente do portfolio 1, o segundo e o terceiro apresentam somente os id's. Será necessário extrair seus dados de market posteriormente.

### 1.1. Dados faltantes

In [247]:
market.isna().sum()

id                              0
fl_matriz                       0
de_natureza_juridica            0
sg_uf                           0
natureza_juridica_macro         0
                            ...  
qt_funcionarios_24meses    358724
tx_crescimento_12meses     388410
tx_crescimento_24meses     388284
tx_rotatividade            358724
qt_filiais                      0
Length: 181, dtype: int64

Existe uma grande quantidade de missing data. Então, vamos manter apenas as colunas com menos de 15% de dados faltantes.

In [250]:
market_missing = market.isna().sum()

In [251]:
cols = []
[cols.append(col) for col in market.columns if market_missing[col] / len(market) < 0.15]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [252]:
new_market = market[cols]

In [253]:
new_market.isna().sum()

id                                         0
fl_matriz                                  0
de_natureza_juridica                       0
sg_uf                                      0
natureza_juridica_macro                    0
de_ramo                                    0
setor                                   1927
idade_empresa_anos                         0
idade_emp_cat                              0
fl_me                                      0
fl_sa                                      0
fl_epp                                     0
fl_mei                                     0
fl_ltda                                    0
dt_situacao                                6
fl_st_especial                             0
fl_email                                   0
fl_telefone                                0
fl_rm                                      0
nm_divisao                              1927
nm_segmento                             1927
fl_spa                                  1927
fl_antt   

Agora, podemos remover as linhas que aprensetam dados faltantes sem perder tanto de nosso dataset.

In [254]:
new_market = new_market.dropna()

In [255]:
new_market.shape

(364549, 40)

In [256]:
new_market.isna().sum()

id                                     0
fl_matriz                              0
de_natureza_juridica                   0
sg_uf                                  0
natureza_juridica_macro                0
de_ramo                                0
setor                                  0
idade_empresa_anos                     0
idade_emp_cat                          0
fl_me                                  0
fl_sa                                  0
fl_epp                                 0
fl_mei                                 0
fl_ltda                                0
dt_situacao                            0
fl_st_especial                         0
fl_email                               0
fl_telefone                            0
fl_rm                                  0
nm_divisao                             0
nm_segmento                            0
fl_spa                                 0
fl_antt                                0
fl_veiculo                             0
vl_total_veiculo

## 2.0. Seleção de variáveis

Ao observar a [descrição das variáveis](https://s3-us-west-1.amazonaws.com/codenation-challenges/ml-leads/features_dictionary.pdf), podemos notar que existem algumas que são redundantes e algumas que não aparentam acrescentar nada ao modelo. Vamos removê-las.

In [257]:
new_market.drop(['idade_emp_cat', 'dt_situacao', 
                 'de_faixa_faturamento_estimado',
                 'de_faixa_faturamento_estimado_grupo',
                 'fl_email', 'fl_telefone',
                 'fl_matriz', 'vl_total_veiculos_pesados_grupo',
                 'vl_total_veiculos_leves_grupo', 'nu_meses_rescencia',
                 'vl_faturamento_estimado_grupo_aux'], axis=1, inplace=True)

In [258]:
new_market.columns

Index(['id', 'de_natureza_juridica', 'sg_uf', 'natureza_juridica_macro',
       'de_ramo', 'setor', 'idade_empresa_anos', 'fl_me', 'fl_sa', 'fl_epp',
       'fl_mei', 'fl_ltda', 'fl_st_especial', 'fl_rm', 'nm_divisao',
       'nm_segmento', 'fl_spa', 'fl_antt', 'fl_veiculo', 'sg_uf_matriz',
       'de_saude_tributaria', 'de_saude_rescencia', 'de_nivel_atividade',
       'fl_simples_irregular', 'nm_meso_regiao', 'nm_micro_regiao',
       'fl_passivel_iss', 'vl_faturamento_estimado_aux', 'qt_filiais'],
      dtype='object')

In [259]:
new_market.shape

(364549, 29)

## 3.0. Pré-processamento de dados

### 3.1. Transformando variáveis booleanas

In [260]:
new_market.dtypes

id                              object
de_natureza_juridica            object
sg_uf                           object
natureza_juridica_macro         object
de_ramo                         object
setor                           object
idade_empresa_anos             float64
fl_me                             bool
fl_sa                             bool
fl_epp                            bool
fl_mei                            bool
fl_ltda                           bool
fl_st_especial                    bool
fl_rm                           object
nm_divisao                      object
nm_segmento                     object
fl_spa                          object
fl_antt                         object
fl_veiculo                      object
sg_uf_matriz                    object
de_saude_tributaria             object
de_saude_rescencia              object
de_nivel_atividade              object
fl_simples_irregular            object
nm_meso_regiao                  object
nm_micro_regiao          

In [261]:
for col in new_market.select_dtypes(include=['object']):
    if new_market.select_dtypes(include=['object'])[col].nunique() == 2:
        print(col)

fl_rm
fl_spa
fl_antt
fl_veiculo
fl_simples_irregular
fl_passivel_iss


Ao analisar um pouco mais nossas variáveis categóricas, percebemos que algumas são binárias. Podemos convertê-las para booleanas.

In [262]:
new_market['fl_rm']

0         SIM
1         SIM
2         SIM
3         SIM
4         SIM
         ... 
462290    SIM
462291    SIM
462293    NAO
462294    SIM
462295    SIM
Name: fl_rm, Length: 364549, dtype: object

In [263]:
new_market.replace({'fl_rm': {'SIM': True, 'NAO': False}}, inplace=True)

In [264]:
new_market['fl_spa'] = new_market['fl_spa'].astype('bool')
new_market['fl_antt'] = new_market['fl_spa'].astype('bool')
new_market['fl_veiculo'] = new_market['fl_spa'].astype('bool')
new_market['fl_simples_irregular'] = new_market['fl_spa'].astype('bool')
new_market['fl_passivel_iss'] = new_market['fl_spa'].astype('bool')
new_market['fl_rm'] = new_market['fl_spa'].astype('bool')

### 3.2. LabelEncoder

Agora, podemos transformar nossas variáveis categóricas em números.

Selecionando apenas as variáveis categóricas para efetivamente transformá-las em categóricas:

In [267]:
object_cols = new_market.select_dtypes(include='object').columns[1:]

In [268]:
for col in object_cols:
    new_market[col] = new_market[col].astype('category')

In [269]:
cat_cols = new_market.select_dtypes(include=['category']).columns

In [271]:
encoded_market = new_market.copy()

In [272]:
le = LabelEncoder()

for col in non_numerical_cols:
    encoded_market[col] = le.fit_transform(encoded_market[col])

In [273]:
encoded_market.head()

,id,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,fl_me,fl_sa,fl_epp,...,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,fl_simples_irregular,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,vl_faturamento_estimado_aux,qt_filiais
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,57,4,2,11,2,14.457534,0,0,0,...,19,4,0,0,0,6,48,0,3132172.8,0
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,16,3,5,20,4,1.463014,0,0,0,...,16,2,1,1,0,4,69,0,210000.0,0
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,16,1,5,31,4,7.093151,0,0,0,...,2,0,0,2,0,2,43,0,50000.0,0
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,16,1,5,24,4,6.512329,0,0,0,...,2,0,0,2,0,2,43,0,210000.0,0
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,16,4,5,26,4,3.200000,0,0,0,...,19,4,0,0,0,6,48,0,210000.0,0


In [274]:
encoded_market.dtypes

id                              object
de_natureza_juridica             int32
sg_uf                            int32
natureza_juridica_macro          int32
de_ramo                          int32
setor                            int32
idade_empresa_anos             float64
fl_me                            int64
fl_sa                            int64
fl_epp                           int64
fl_mei                           int64
fl_ltda                          int64
fl_st_especial                   int64
fl_rm                            int64
nm_divisao                       int32
nm_segmento                      int32
fl_spa                           int64
fl_antt                          int64
fl_veiculo                       int64
sg_uf_matriz                     int32
de_saude_tributaria              int32
de_saude_rescencia               int32
de_nivel_atividade               int32
fl_simples_irregular             int64
nm_meso_regiao                   int32
nm_micro_regiao          

### 3.3. StandardScaler

In [283]:
from sklearn.preprocessing import StandardScaler

numerical_cols = encoded_market.select_dtypes(include=['float64']).columns

scaler = StandardScaler()
scaler.fit(encoded_market[numerical_cols])
encoded_market[numerical_cols] = scaler.transform(encoded_market[numerical_cols])

In [285]:
encoded_market.head()

,id,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,fl_me,fl_sa,fl_epp,...,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,fl_simples_irregular,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,vl_faturamento_estimado_aux,qt_filiais
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,57,4,2,11,2,0.387076,0,0,0,...,19,4,0,0,0,6,48,0,0.068293,0
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,16,3,5,20,4,-0.983356,0,0,0,...,16,2,1,1,0,4,69,0,-0.018665,0
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,16,1,5,31,4,-0.389589,0,0,0,...,2,0,0,2,0,2,43,0,-0.023427,0
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,16,1,5,24,4,-0.450843,0,0,0,...,2,0,0,2,0,2,43,0,-0.018665,0
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,16,4,5,26,4,-0.800169,0,0,0,...,19,4,0,0,0,6,48,0,-0.018665,0


## 4.0. Extração de variáveis

Como a grande maioria de nossas features é categórica, usaremos o MCA (parecido com o PCA, mas mais específico para variáveis categóricas).

In [278]:
mca = MCA()

mca_market = mca.fit(encoded_market[cat_cols])
mca_market = mca.transform(encoded_market[cat_cols])

mca_market

,0,1
0,-0.074034,0.380295
1,0.611583,0.160659
2,-0.136054,-0.426209
3,-0.092588,-0.370683
4,-0.045096,-0.312638
...,...,...
462290,-0.392001,0.060766
462291,0.138645,-0.243006
462293,-0.180466,-0.055392
462294,0.109384,-0.247085


Agora, unimos os dados gerados pelo MCA e descartamos as variáveis categóricas.

In [286]:
market_transformed = encoded_market.join(mca_market)

In [287]:
market_transformed = market_transformed.drop(cat_cols, axis=1)

In [288]:
market_transformed.shape

(364549, 18)

In [289]:
market_transformed.head()

,id,idade_empresa_anos,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,fl_st_especial,fl_rm,fl_spa,fl_antt,fl_veiculo,fl_simples_irregular,fl_passivel_iss,vl_faturamento_estimado_aux,qt_filiais,0,1
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,0.387076,0,0,0,0,0,0,0,0,0,0,0,0,0.068293,0,-0.074034,0.380295
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,-0.983356,0,0,0,1,0,0,0,0,0,0,0,0,-0.018665,0,0.611583,0.160659
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,-0.389589,0,0,0,1,0,0,0,0,0,0,0,0,-0.023427,0,-0.136054,-0.426209
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,-0.450843,0,0,0,0,0,0,0,0,0,0,0,0,-0.018665,0,-0.092588,-0.370683
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,-0.800169,0,0,0,0,0,0,0,0,0,0,0,0,-0.018665,0,-0.045096,-0.312638


## 5.0. Corrigindo portfolios

### Portfolio 1

In [290]:
portfolio_1.head()

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,...,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
0,dabe79bec87c88ae04e869bf6bd321ee5e1893cecf6625...,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,COMERCIO VAREJISTA,COMERCIO,0.646575,<= 1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,32e5f4e10932153a7ba869cb0386e7e02d49d2461046b8...,False,EMPRESARIO INDIVIDUAL,MA,OUTROS,COMERCIO VAREJISTA,COMERCIO,6.586301,5 a 10,False,...,NaN,NaN,NaN,0.0,0.0,2.0,NaN,-100.0,0.0,4
2,a95d6f30bba445bd3d6b0c5b36f865b38ec01d17336090...,True,EMPRESARIO INDIVIDUAL,MA,OUTROS,COMERCIO VAREJISTA,COMERCIO,8.010959,5 a 10,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,6cb309685cea0b6d2988818792ec2e6fcb2bd02e0afa9e...,True,EMPRESARIO INDIVIDUAL,RN,OUTROS,COMERCIO VAREJISTA,COMERCIO,20.863014,> 20,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,f72aa7fa6787b0a5a1c88885b6120850df8ee0f71adc25...,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,COMERCIO VAREJISTA,COMERCIO,18.726027,15 a 20,False,...,39.454762,91.2,19.333333,14.0,15.0,25.0,-6.666667,-44.0,0.0,3


In [291]:
portfolio_1 = portfolio_1[['id']]

In [292]:
portfolio_1 = pd.merge(portfolio_1, market_transformed, on='id',  how='left')

In [293]:
portfolio_1.shape

(555, 18)

In [296]:
portfolio_1.head()

,id,idade_empresa_anos,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,fl_st_especial,fl_rm,fl_spa,fl_antt,fl_veiculo,fl_simples_irregular,fl_passivel_iss,vl_faturamento_estimado_aux,qt_filiais,0,1
1,32e5f4e10932153a7ba869cb0386e7e02d49d2461046b8...,-0.443042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.018665,4.0,0.187696,-0.017300
2,a95d6f30bba445bd3d6b0c5b36f865b38ec01d17336090...,-0.292795,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.023427,0.0,-0.528122,-0.173585
3,6cb309685cea0b6d2988818792ec2e6fcb2bd02e0afa9e...,1.062612,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.023427,0.0,0.374063,0.028456
4,f72aa7fa6787b0a5a1c88885b6120850df8ee0f71adc25...,0.837241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.065227,3.0,-0.067074,0.442439
5,f7e2894dce5dbb49662da50c55bcb96cdc3d34188abfe8...,-0.479737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.018665,0.0,-0.033695,-0.064701


In [294]:
portfolio_1.isna().sum()

id                               0
idade_empresa_anos             111
fl_me                          111
fl_sa                          111
fl_epp                         111
fl_mei                         111
fl_ltda                        111
fl_st_especial                 111
fl_rm                          111
fl_spa                         111
fl_antt                        111
fl_veiculo                     111
fl_simples_irregular           111
fl_passivel_iss                111
vl_faturamento_estimado_aux    111
qt_filiais                     111
0                              111
1                              111
dtype: int64

In [295]:
portfolio_1 = portfolio_1.dropna()

In [297]:
portfolio_1.shape

(444, 18)

### Portfólio 2

In [298]:
portfolio_2.head()

,id
0,09e95c1a84049001d086470a8f320a19b076f955a89122...
1,dc9d155f5bcd3172031e2ec9431786e8871ff4b0ff4728...
2,16843c9ffb920170477118933798510d8d5f00f5c56c12...
3,ff045934d90aab184dd15e66daf3b3c97193bda5449298...
4,a0ce528caab2c62497a548d0ebd9e935143ef81ed7dbf0...


In [299]:
portfolio_2 = pd.merge(portfolio_2, market_transformed, on='id',  how='left')

In [300]:
portfolio_2.shape

(566, 18)

In [301]:
portfolio_2.isna().sum()

id                              0
idade_empresa_anos             70
fl_me                          70
fl_sa                          70
fl_epp                         70
fl_mei                         70
fl_ltda                        70
fl_st_especial                 70
fl_rm                          70
fl_spa                         70
fl_antt                        70
fl_veiculo                     70
fl_simples_irregular           70
fl_passivel_iss                70
vl_faturamento_estimado_aux    70
qt_filiais                     70
0                              70
1                              70
dtype: int64

In [302]:
portfolio_2 = portfolio_2.dropna()

In [303]:
portfolio_2.shape

(496, 18)

### Portfólio 3

In [304]:
portfolio_3.head()

,id
0,bb61c09449a1c1f8b78955d8b7a7f73599e78358993fe2...
1,16843c9ffb920170477118933798510d8d5f00f5c56c12...
2,1b8d092daff12bed8b3c2ab93bf5df2921865a68aca185...
3,3aafa3a1f12f483740a52956af100223333d0e3c79bf2c...
4,c60431fbca90067f105570e1e8a684bcd7048205bbf6c4...


In [305]:
portfolio_3 = pd.merge(portfolio_3, market_transformed, on='id',  how='left')

In [306]:
portfolio_3.shape

(265, 18)

In [307]:
portfolio_3.isna().sum()

id                              0
idade_empresa_anos             25
fl_me                          25
fl_sa                          25
fl_epp                         25
fl_mei                         25
fl_ltda                        25
fl_st_especial                 25
fl_rm                          25
fl_spa                         25
fl_antt                        25
fl_veiculo                     25
fl_simples_irregular           25
fl_passivel_iss                25
vl_faturamento_estimado_aux    25
qt_filiais                     25
0                              25
1                              25
dtype: int64

In [308]:
portfolio_3 = portfolio_3.dropna()

In [309]:
portfolio_3.shape

(240, 18)

## 6.0. Modelo

Removendo os dados dos portfolios para criação do modelo.

In [310]:
market_transformed = market_transformed[~market_transformed['id'].isin(portfolio_1['id'])]
market_transformed = market_transformed[~market_transformed['id'].isin(portfolio_2['id'])]
market_transformed = market_transformed[~market_transformed['id'].isin(portfolio_3['id'])]
market_transformed.shape

(363466, 18)

In [311]:
market_final = market_transformed.drop('id', axis=1)

Para o modelo, como se trata de um problema não supervisionado, usaremos o Nearest Neighbors. Os parâmetros

In [313]:
from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors=3)

neigh.fit(market_final)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                 radius=1.0)

### 7.0. Recomendações

In [326]:
def recomendacao(portfolio):
    return neigh.kneighbors(portfolio.drop('id', axis=1), return_distance=False)

### 7.1. Portfolio 1

In [327]:
recomendacao_1 = recomendacao(portfolio_1)

In [328]:
leads_1 = market_transformed.iloc[recomendacao_1.reshape(-1)]

In [329]:
leads_1.head()

,id,idade_empresa_anos,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,fl_st_especial,fl_rm,fl_spa,fl_antt,fl_veiculo,fl_simples_irregular,fl_passivel_iss,vl_faturamento_estimado_aux,qt_filiais,0,1
138574,d1222cffd228ae3a9c4e35372e29bcbc363d9800c8aa96...,-0.410970,0,0,0,0,0,0,0,0,0,0,0,0,-0.018665,4,0.182492,-0.014896
79044,ce4bec0689740e4ca4c42fd0f342105abf82948504ae67...,-0.504875,0,0,0,0,0,0,0,0,0,0,0,0,-0.018665,4,0.195093,-0.013364
86782,2cb0b38243044147aa2a9e7ae054c513bb5a48891c77a3...,-0.504875,0,0,0,0,0,0,0,0,0,0,0,0,-0.018665,4,0.195093,-0.013364
118303,ec5fced14445fe51bd9dbcc5e2484d1c323abad7e681b5...,-0.291639,0,0,0,1,0,0,0,0,0,0,0,0,-0.018665,0,-0.528122,-0.173585
186523,afcf25c67e3176fbc5bb8eef5051d083ef737ea6a8f78d...,-0.290483,0,0,0,1,0,0,0,0,0,0,0,0,-0.018665,0,-0.528122,-0.173585


In [330]:
leads_1.shape

(1332, 18)

### 7.2. Portfolio 2

In [331]:
recomendacao_2 = recomendacao(portfolio_2)

In [332]:
leads_2 = market_transformed.iloc[recomendacao_2.reshape(-1)]

In [333]:
leads_2.head()

,id,idade_empresa_anos,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,fl_st_especial,fl_rm,fl_spa,fl_antt,fl_veiculo,fl_simples_irregular,fl_passivel_iss,vl_faturamento_estimado_aux,qt_filiais,0,1
13367,1d43297a375fb9237831adcdf133256c6777146fa63b93...,-0.100940,0,0,0,0,0,0,0,0,0,0,0,0,10.663249,0,0.358374,0.736335
326831,9a8c308ac607762799665f39f32071738d95eb184b7687...,-0.134746,0,1,0,0,0,0,0,0,0,0,0,0,10.080452,1,-0.168638,0.265494
388297,4db3114a4f39babd5fcceb9a99b7f872ad579782cc249c...,0.203023,0,0,0,0,0,0,0,0,0,0,0,0,10.710773,1,-0.300030,0.069002
47385,d039506a9af446f69f3c8af41ce08b5e6aec95f89a40ef...,3.487377,0,0,0,0,0,0,0,0,0,0,0,0,4.709028,0,-0.051939,0.629299
22688,fe595499889f866ed9380cbe338ad3362c9bc20618f435...,3.577526,0,0,0,0,0,0,0,0,0,0,0,0,4.721293,0,0.029255,0.776519


In [334]:
leads_2.shape

(1488, 18)

### 7.3. Portfolio 3

In [335]:
recomendacao_3 = recomendacao(portfolio_3)

In [336]:
leads_3 = market_transformed.iloc[recomendacao_3.reshape(-1)]

In [337]:
leads_3.head()

,id,idade_empresa_anos,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,fl_st_especial,fl_rm,fl_spa,fl_antt,fl_veiculo,fl_simples_irregular,fl_passivel_iss,vl_faturamento_estimado_aux,qt_filiais,0,1
218745,d859dad169c53ae868b52d1d912c01d594797f53465f66...,1.583278,0,0,0,0,0,0,0,0,0,0,0,0,4.390162,2,-0.131084,0.545850
119895,8e766235801b7dd152a9bac144950e2918a95c97a6c44c...,1.066368,0,0,0,0,0,0,0,0,0,0,0,0,4.871528,2,-0.062886,0.457293
423041,535295e415f6a6e0929adab42401c9ca569843005a85b3...,0.301261,0,0,0,0,0,0,0,0,0,0,0,0,4.473098,2,-0.198320,0.321716
400016,c18382bf7178d8d4449972c667d648c108182c5d22df8b...,0.430705,0,0,0,0,0,0,0,0,0,0,0,0,1.016001,0,-0.624455,0.307269
413133,76474a7ddc6b52aad04e84553ddb856c774ddc465804ae...,0.730045,0,0,0,0,0,0,0,0,0,0,0,0,0.951615,0,-0.262158,0.189117


In [338]:
leads_3.shape

(720, 18)